In [343]:
import pandas as pd
import numpy as np
import requests
import swifter
from bs4 import BeautifulSoup
import logging
logger = logging.getLogger('test')

In [252]:
def create_col_names_win_loss_round():
    """
    Generates column names for significant strike statistics.

    Returns:
        list: A list of strings, each representing a column name for a specific significant strike statistic.
    """
    col_names = []
    fighters = ['fighter-a', 'fighter-b']
    fight_rounds = ['3R', '5R']
    win_loss = ['wins', 'losses']
    rounds = ['R1', 'R2', 'R3', 'R4', 'R5', 'decision', 'overall']
    time_periods = ['alltime', '1y']
    for fighter in fighters:
        for num_rounds in fight_rounds:
            for wl in win_loss:
                for dec_round in rounds:
                    for time_period in time_periods:
                        if num_rounds == '3R' and (dec_round == 'R4' or dec_round == 'R5'):
                            continue
                        col_name = f"{fighter.replace(' ', '_')}_{num_rounds.replace(' ', '_')}_{dec_round}_{wl}_{time_period.replace(' ', '_')}"
                        col_names.append(col_name)
    return col_names

In [325]:
col = create_col_names_win_loss_round()
three_round_fight_cols_a = col[:48][:20]
five_round_fight_cols_a = col[:48][20:]
# print(col)
print(three_round_fight_cols_a) # 3R
print(five_round_fight_cols_a) # 5R
print(len(col))

['fighter-a_3R_R1_wins_alltime', 'fighter-a_3R_R1_wins_1y', 'fighter-a_3R_R2_wins_alltime', 'fighter-a_3R_R2_wins_1y', 'fighter-a_3R_R3_wins_alltime', 'fighter-a_3R_R3_wins_1y', 'fighter-a_3R_decision_wins_alltime', 'fighter-a_3R_decision_wins_1y', 'fighter-a_3R_overall_wins_alltime', 'fighter-a_3R_overall_wins_1y', 'fighter-a_3R_R1_losses_alltime', 'fighter-a_3R_R1_losses_1y', 'fighter-a_3R_R2_losses_alltime', 'fighter-a_3R_R2_losses_1y', 'fighter-a_3R_R3_losses_alltime', 'fighter-a_3R_R3_losses_1y', 'fighter-a_3R_decision_losses_alltime', 'fighter-a_3R_decision_losses_1y', 'fighter-a_3R_overall_losses_alltime', 'fighter-a_3R_overall_losses_1y']
['fighter-a_5R_R1_wins_alltime', 'fighter-a_5R_R1_wins_1y', 'fighter-a_5R_R2_wins_alltime', 'fighter-a_5R_R2_wins_1y', 'fighter-a_5R_R3_wins_alltime', 'fighter-a_5R_R3_wins_1y', 'fighter-a_5R_R4_wins_alltime', 'fighter-a_5R_R4_wins_1y', 'fighter-a_5R_R5_wins_alltime', 'fighter-a_5R_R5_wins_1y', 'fighter-a_5R_decision_wins_alltime', 'fighter-a_

In [254]:
def get_unique_outcome_methods(column):
    """
    Returns a list of unique strings from the provided column.

    Args:
    column (pd.Series): A Pandas Series object representing a DataFrame column.

    Returns:
    list: A list containing unique strings found in the column.
    """
    return column.unique().tolist()


In [331]:
# creating the leon edwards allfights csv


df = pd.read_csv('ufc_men_stats_by_fight_td_col_rename.csv', encoding='ISO-8859-1')
# Assuming df is your DataFrame
# Calculate the mode of the 'outcome_format' column
# mode_outcome_format = df['outcome_format'].mode()[0]
# print(mode_outcome_format)
# # # Replace values that are not '5-5-5' or '5-5-5-5-5' with the mode
# df['outcome_format'] = df['outcome_format'].apply(lambda x: x if x == "(5-5-5)" or x == "(5-5-5-5-5)" else mode_outcome_format)

# df= df.loc[:1000-1]
# fighter_a_id_vals = df.outcome_format.values

# outcome_format_values = df.outcome_format.values
fighter_id = 'f1fac969a1d70b08'
# print(fight_format_values)
fighter_a_id_vals = df.fighter_a_id.values
fighter_b_id_vals = df.fighter_b_id.values

allfights = df[((fighter_a_id_vals == fighter_id) | (fighter_b_id_vals == fighter_id))]
allfights = allfights.reset_index(drop=True) # resseting index other wise has index of original df

allfights.to_csv('leon_edwards_allfights.csv', index=True)


df_test = pd.read_csv('leon_edwards_allfights.csv', encoding='ISO-8859-1')
print(len(df_test))
df_test.apply(lambda x: print(x.name), axis=1)   




16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
dtype: object

In [256]:
def compute_year_ago(df):
    temp_year_ago_df = pd.to_datetime(df['date'])
    temp_year_ago_df = temp_year_ago_df - pd.DateOffset(years=1)
    return temp_year_ago_df

In [276]:
def get_fighter_fights_win_loss_round(df, fighter_id, index):
        """
        Retrieves the most recent fights of a fighter before a given index in the dataframe.

        Args:
            df (pd.DataFrame): The dataframe containing fight records.
            fighter_id (str): The ID of the fighter.
            index (int): The index in the dataframe up to which fights should be considered.

        Returns:
            tuple: Contains three dataframes - the last 3 fights, the last 5 fights, and all previous fights.
        """
        year_ago = compute_year_ago(df)
        all_prev_fights = df.loc[:index-1]

        if not all_prev_fights.empty:
            # Filter fights involving the specified fighter
            fighter_a_id_vals = all_prev_fights.fighter_a_id.values
            fighter_b_id_vals = all_prev_fights.fighter_b_id.values

            prev_fights = all_prev_fights[((fighter_a_id_vals == fighter_id) | (fighter_b_id_vals == fighter_id))]
            # prev_fights_last_year = all_prev_fights[((fighter_a_id_vals == fighter_id) | (fighter_b_id_vals == fighter_id)) & (pd.to_datetime(all_prev_fights['date']) > year_ago[index])]


            if not prev_fights.empty:
                outcome_format_values = prev_fights.outcome_format.values

                # Filter fights that are 3R or 5R
                all_prev_fights_3R = prev_fights[outcome_format_values == "(5-5-5)"]
                all_prev_fights_5R = prev_fights[outcome_format_values == "(5-5-5-5-5)"]
                
                return all_prev_fights_3R, all_prev_fights_5R, year_ago[index]
                
            else:
                 return pd.DataFrame(), pd.DataFrame(), year_ago[index]
            
            
    
        return pd.DataFrame(), pd.DataFrame(), year_ago[index]


    

In [258]:
def get_wins_losses(df, fighter_id):
    """
    Retrieves the wins and losses of a fighter.

    Args:
        df (pd.DataFrame): The dataframe containing fight records.
        fighter_id (str): The ID of the fighter.
        colnames (list): A list of column names to be used for the dataframe.

    Returns:
        tuple: Contains two dataframes - one containing the wins of the fighter, and the other containing the losses of the fighter.
    """
   
    if df.empty:
        return pd.DataFrame(), pd.DataFrame()
    
    fighter_wins = df[(df['winner_id'] == fighter_id)]
    fighter_losses = df[(df['winner_id'] != fighter_id)]

    assert len(fighter_wins) + len(fighter_losses) == len(df)

    return fighter_wins, fighter_losses
    

In [259]:
def isDecision(outcome_methods):
    return outcome_methods.str.contains("Decision", case=False, na=False)

In [321]:
def compute_win_loss_round_details_alltime(fights_df, round_num):

    if fights_df.empty:
        return pd.DataFrame()
    
    round_outcome_fights = fights_df[(fights_df['outcome_round'] == round_num) & (~isDecision(fights_df['outcome_method']))]
    # print(round_outcome_fights)
    return round_outcome_fights

In [340]:
def get_win_loss_round_details(fights_won, fights_lost, fighter_id, colnames, year_ago):
    # fights_won_3R, fights_lost_3R, fighter_id, colnames[:20], year_ago, index
    res = []
    # if len(colnames) == 2:
    #     print(colnames)
    
    # print(year_ago[index])
    for i in range(0, len(colnames), 2):

        split = colnames[i].split('_')
        round_num = split[2]
        outcome = split[3]
        
        if outcome == 'wins':
            outcome = 1
        else:
            outcome = 0

        # print(index)
        if round_num == 'overall':
            if outcome:
                res.append(len(fights_won))
                res.append(len(fights_won[(pd.to_datetime(fights_won['date'])> year_ago)]))
            else:
                res.append(len(fights_lost))
                res.append(len(fights_lost[(pd.to_datetime(fights_lost['date']) > year_ago)]))
            continue
            

        if round_num == 'decision':
            if outcome:
                res.append(len(fights_won[isDecision(fights_won['outcome_method'])]))
                res.append(len(fights_won[isDecision(fights_won['outcome_method']) & (pd.to_datetime(fights_won['date']) > year_ago)]))
                
            else:
                res.append(len(fights_lost[isDecision(fights_lost['outcome_method'])]))
                res.append(len(fights_lost[isDecision(fights_lost['outcome_method']) & (pd.to_datetime(fights_lost['date']) > year_ago)]))
            continue
            
        elif outcome:
            round_outcome_fights = compute_win_loss_round_details_alltime(fights_won, round_num)
            # print(f'printing the date col: {round_outcome_fights["date"]}')
            # print(round_outcome_fights.columns)
            res.append(len(round_outcome_fights))
            res.append(len(round_outcome_fights[(pd.to_datetime(round_outcome_fights['date']) > year_ago)]))
            continue
        else:
            round_outcome_fights = compute_win_loss_round_details_alltime(fights_lost, round_num)
            # print(round_outcome_fights.columns)
            res.append(len(round_outcome_fights))
            res.append(len(round_outcome_fights[(pd.to_datetime(round_outcome_fights['date']) > year_ago)]))
            continue


    # print(len(res))
    return res

In [341]:
def get_win_loss_round_feats(df, fighter_id, index, colnames):

    res = []
    all_prev_fights_3R, all_prev_fights_5R, year_ago= get_fighter_fights_win_loss_round(df, fighter_id, index)

    fights_won_3R, fights_lost_3R = get_wins_losses(all_prev_fights_3R, fighter_id)
    fights_won_5R, fights_lost_5R = get_wins_losses(all_prev_fights_5R, fighter_id)
    # print(index)
    res.append(get_win_loss_round_details(fights_won_3R, fights_lost_3R, fighter_id, colnames[:20], year_ago))
    res.append(get_win_loss_round_details(fights_won_5R, fights_lost_5R, fighter_id, colnames[20:], year_ago))
    # print(len(res)) # 2
    return res




In [344]:
def calculate_win_loss_round(df, fighter_a_id, fighter_b_id, index, colnames):
    res = []
    logging.info(f'index: {index}')
    print(f' this is the index: {index}')
    # print(f'cols for a: {colnames[:48]}')
    # print(f'cols for b: {colnames[48:]}')
    # assert len(colnames[:48]) + len(colnames[48:]) == len(colnames) == 96
    res.append(get_win_loss_round_feats(df, fighter_a_id, index, colnames[:48])) # has all of fighter_a's cols
    res.append(get_win_loss_round_feats(df, fighter_b_id, index, colnames[48:])) # has all of fighter_b's cols
    # print(len(res)) # 2
    return pd.Series(res)

In [333]:
def create_win_loss_round_feats(df):
    """
    Creates a dataframe with added features for significant strikes and their differentials.

    Args:
        df (pd.DataFrame): The original dataframe containing fight data.

    Returns:
        pd.DataFrame: A dataframe with additional columns for significant strike features and differentials.
    """
    
    # Generate column names for significant strikes and differentials
    col_names = create_col_names_win_loss_round()
    # print(len(col_names))
    df['date'] = pd.to_datetime(df['date'])
    # Copy the input dataframe and calculate significant strike features
    input_df = df.copy()
    result_features = pd.DataFrame(columns=col_names)
    result_features[col_names] = input_df.swifter.progress_bar(True).apply(lambda row: calculate_win_loss_round(input_df, row['fighter_a_id'], row['fighter_b_id'], row.name, col_names), axis=1)

    # Combine the input dataframe with the calculated features
    result_df = pd.concat([input_df, result_features], axis=1)

    return result_df

In [350]:
leon_edwards_df = pd.read_csv('leon_edwards_allfights.csv', encoding='ISO-8859-1')

result_df = create_win_loss_round_feats(leon_edwards_df)

# result_df.to_csv('leon_edwards_win_loss_round.csv', index=False)



KeyError: 'date'

In [348]:

fighter_id = 'f1fac969a1d70b08' # leon edward id
index = 15 
leon_edwards_df = pd.read_csv('leon_edwards_allfights.csv', encoding='ISO-8859-1')
all_prev_fights_3R, all_prev_fights_5R, year_ago = get_fighter_fights_win_loss_round(leon_edwards_df, fighter_id, 15)
fights_won_3R, fights_lost_3R = get_wins_losses(all_prev_fights_3R, fighter_id)
fights_won_5R, fights_lost_5R = get_wins_losses(all_prev_fights_5R, fighter_id)
colnames = create_col_names_win_loss_round()[:48]
# # print(colnames[:20])
print(len(get_win_loss_round_details(fights_won_3R, fights_lost_3R, fighter_id, colnames[:20], year_ago)))
print(len(get_win_loss_round_details(fights_won_5R, fights_lost_5R, fighter_id, colnames[20:], year_ago)))



20
28
